In [1]:
import pandas as pd
from sqlalchemy import create_engine
from secret import username, password

In [2]:
#importing videogame database
vgdf = pd.read_csv("vgsales.csv")
vgdf

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [3]:
#removing older platforms of which we can't gather reviews online per Tanner
# Dropping null values from df
to_drop = ['NES', 'SNES', 'SAT', '2600', 'GB', 'GEN', 'NG', 'SCD', 'WS', '3DO', 'TG16', 'GG', 'PCFX']
vgdf_plat_clean = vgdf[~vgdf['Platform'].isin(to_drop)]
vgdf_plat_clean = vgdf_plat_clean.dropna()
vgdf_plat_clean = vgdf_plat_clean.drop_duplicates(subset=['Name', 'Platform'], keep='first')
vgdf_plat_clean.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15507 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          15507 non-null  int64  
 1   Name          15507 non-null  object 
 2   Platform      15507 non-null  object 
 3   Year          15507 non-null  float64
 4   Genre         15507 non-null  object 
 5   Publisher     15507 non-null  object 
 6   NA_Sales      15507 non-null  float64
 7   EU_Sales      15507 non-null  float64
 8   JP_Sales      15507 non-null  float64
 9   Other_Sales   15507 non-null  float64
 10  Global_Sales  15507 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


#### Publisher ID Table

In [4]:
#creating publisher table with id determined by its index #
vg_publisher_df = pd.DataFrame({'publisher':vgdf_plat_clean['Publisher'].unique()})
vg_publisher_df.index.name='publisher_id'
vg_publisher_df

,publisher
publisher_id,
0,Nintendo
1,Microsoft Game Studios
2,Take-Two Interactive
3,Sony Computer Entertainment
4,Activision
...,...
519,Inti Creates
520,Takuyo
521,Interchannel-Holon


In [5]:
vg_publisher_df.dropna(subset=['publisher'], inplace=True)
vg_publisher_df

,publisher
publisher_id,
0,Nintendo
1,Microsoft Game Studios
2,Take-Two Interactive
3,Sony Computer Entertainment
4,Activision
...,...
519,Inti Creates
520,Takuyo
521,Interchannel-Holon


#### Platform ID Table

In [6]:
#creating platform table with id determined by its index #
vg_platform_df = pd.DataFrame({'platform_name':vgdf_plat_clean['Platform'].unique()})
vg_platform_df.index.name='id'
vg_platform_df

,platform_name
id,
0,Wii
1,DS
2,X360
3,PS3
4,PS2
5,GBA
6,3DS
7,PS4
8,N64


#### Genre ID Table

In [7]:
vg_genre_df = pd.DataFrame({'genre':vgdf_plat_clean['Genre'].unique()})
vg_genre_df.index.name='id'
vg_genre_df

,genre
id,
0,Sports
1,Racing
2,Platform
3,Misc
4,Simulation
5,Action
6,Role-Playing
7,Puzzle
8,Shooter


### Replace platform names with id's - TEST

In [8]:
#Test
#Creating dictionary with platform_id and platform - then inversing key/value to format a dictionary that can then be used to
#replace platform with id in other cleaned tables

platformdict = vg_platform_df.to_dict('index')
platformdict

for key in platformdict:
    platformdict[key] = platformdict[key]["platform_name"]
    
platformdict
    
platformdict_s = {value:key for key, value in platformdict.items()}
platformdict_s    

{'Wii': 0,
 'DS': 1,
 'X360': 2,
 'PS3': 3,
 'PS2': 4,
 'GBA': 5,
 '3DS': 6,
 'PS4': 7,
 'N64': 8,
 'PS': 9,
 'XB': 10,
 'PC': 11,
 'PSP': 12,
 'XOne': 13,
 'GC': 14,
 'WiiU': 15,
 'DC': 16,
 'PSV': 17}

In [9]:
#test platform_id replace in Platform column 
vgdf_replacer = {
    "Platform": platformdict_s
}
vgdf.replace(vgdf_replacer)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,0,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,0,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,0,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,5,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,14,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,4,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,1,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


### Add id's to tables formatted by Tyler - Game Title, Sales

#### Game Title

In [10]:
#import transformed game title table in for publisher id transformation
game_title_df = pd.read_csv('title.csv')
game_title_df = game_title_df.set_index('title_id')
game_title_df

,game_title,year_released,publisher,genre
title_id,,,,
0,Wii Sports,2006.0,Nintendo,Sports
1,Mario Kart Wii,2008.0,Nintendo,Racing
2,Wii Sports Resort,2009.0,Nintendo,Sports
3,New Super Mario Bros.,2006.0,Nintendo,Platform
4,Wii Play,2006.0,Nintendo,Misc
...,...,...,...,...
10659,Chou Ezaru wa Akai Hana: Koi wa Tsuki ni Shiru...,2016.0,dramatic create,Action
10660,Eiyuu Densetsu: Sora no Kiseki Material Collec...,2007.0,Falcom Corporation,Role-Playing
10661,Plushees,2008.0,Destineer,Simulation


In [11]:
#create publisher id dict for replacing in publisher column of table
#use vg_publisher_df from before

publisherdict = vg_publisher_df.to_dict('index')
publisherdict

for key in publisherdict:
    publisherdict[key] = publisherdict[key]["publisher"]
    
publisherdict_s = {value:key for key, value in publisherdict.items()}
publisherdict_s  

{'Nintendo': 0,
 'Microsoft Game Studios': 1,
 'Take-Two Interactive': 2,
 'Sony Computer Entertainment': 3,
 'Activision': 4,
 'Ubisoft': 5,
 'Bethesda Softworks': 6,
 'Electronic Arts': 7,
 'Sega': 8,
 'SquareSoft': 9,
 '505 Games': 10,
 'GT Interactive': 11,
 'Konami Digital Entertainment': 12,
 'Sony Computer Entertainment Europe': 13,
 'Square Enix': 14,
 'LucasArts': 15,
 'Virgin Interactive': 16,
 'Warner Bros. Interactive Entertainment': 17,
 'Capcom': 18,
 'Universal Interactive': 19,
 'Eidos Interactive': 20,
 'RedOctane': 21,
 'Atari': 22,
 'Vivendi Games': 23,
 'Enix Corporation': 24,
 'Namco Bandai Games': 25,
 'Hasbro Interactive': 26,
 'THQ': 27,
 'Fox Interactive': 28,
 'Acclaim Entertainment': 29,
 'MTV Games': 30,
 'Disney Interactive Studios': 31,
 'Majesco Entertainment': 32,
 'Codemasters': 33,
 'Red Orb': 34,
 'Level 5': 35,
 'Midway Games': 36,
 'JVC': 37,
 'Deep Silver': 38,
 '989 Studios': 39,
 'NCSoft': 40,
 'UEP Systems': 41,
 'Maxis': 42,
 'Tecmo Koei': 43,


In [12]:
genredict = vg_genre_df.to_dict('index')

for key in genredict:
    genredict[key] = genredict[key]["genre"]
    
genredict_s = {value:key for key, value in genredict.items()}
genredict_s  

{'Sports': 0,
 'Racing': 1,
 'Platform': 2,
 'Misc': 3,
 'Simulation': 4,
 'Action': 5,
 'Role-Playing': 6,
 'Puzzle': 7,
 'Shooter': 8,
 'Fighting': 9,
 'Adventure': 10,
 'Strategy': 11}

In [13]:
#replace publisher w/ publisher_id and genre_id in game title df
pub_id_replacer = {
    "publisher": publisherdict_s,
    "genre": genredict_s
}
game_title_df = game_title_df.replace(pub_id_replacer)
game_title_df = game_title_df.rename(columns={'publisher':'publisher_id', 'genre':'genre_id'})
game_title_df

,game_title,year_released,publisher_id,genre_id
title_id,,,,
0,Wii Sports,2006.0,0,0
1,Mario Kart Wii,2008.0,0,1
2,Wii Sports Resort,2009.0,0,0
3,New Super Mario Bros.,2006.0,0,2
4,Wii Play,2006.0,0,3
...,...,...,...,...
10659,Chou Ezaru wa Akai Hana: Koi wa Tsuki ni Shiru...,2016.0,440,5
10660,Eiyuu Densetsu: Sora no Kiseki Material Collec...,2007.0,215,6
10661,Plushees,2008.0,135,4


In [14]:
game_title_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10664 entries, 0 to 10663
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   game_title     10664 non-null  object 
 1   year_released  10664 non-null  float64
 2   publisher_id   10664 non-null  int64  
 3   genre_id       10664 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 416.6+ KB


In [15]:
game_title_df = game_title_df.astype({'year_released':int})

In [16]:
game_title_df.index.name = "id"
game_title_df

,game_title,year_released,publisher_id,genre_id
id,,,,
0,Wii Sports,2006,0,0
1,Mario Kart Wii,2008,0,1
2,Wii Sports Resort,2009,0,0
3,New Super Mario Bros.,2006,0,2
4,Wii Play,2006,0,3
...,...,...,...,...
10659,Chou Ezaru wa Akai Hana: Koi wa Tsuki ni Shiru...,2016,440,5
10660,Eiyuu Densetsu: Sora no Kiseki Material Collec...,2007,215,6
10661,Plushees,2008,135,4


#### Sales Table

In [17]:
#import transformed sales table by tyler in for platform to platform_id change
sales_df = pd.read_csv('sales.csv')
sales_df

,title_id,platform_id,na_sales,eu_sales,jp_sales,other_sales
0,0,Wii,41.49,29.02,3.77,8.46
1,1,Wii,15.85,12.88,3.79,3.31
2,2,Wii,15.75,11.01,3.28,2.96
3,3,DS,11.38,9.23,6.50,2.90
4,4,Wii,14.03,9.20,2.93,2.85
...,...,...,...,...,...,...
15502,10662,GBA,0.01,0.00,0.00,0.00
15503,2615,GC,0.01,0.00,0.00,0.00
15504,6031,PS2,0.00,0.00,0.00,0.00
15505,10663,DS,0.00,0.01,0.00,0.00


In [18]:
#reference previous code from testing section earlier
plat_replacer = {
    "platform_id": platformdict_s
}

sales_df = sales_df.replace(plat_replacer)
sales_df

,title_id,platform_id,na_sales,eu_sales,jp_sales,other_sales
0,0,0,41.49,29.02,3.77,8.46
1,1,0,15.85,12.88,3.79,3.31
2,2,0,15.75,11.01,3.28,2.96
3,3,1,11.38,9.23,6.50,2.90
4,4,0,14.03,9.20,2.93,2.85
...,...,...,...,...,...,...
15502,10662,5,0.01,0.00,0.00,0.00
15503,2615,14,0.01,0.00,0.00,0.00
15504,6031,4,0.00,0.00,0.00,0.00
15505,10663,1,0.00,0.01,0.00,0.00


In [19]:
sales_df = sales_df.set_index(['title_id','platform_id'])
sales_df

,,na_sales,eu_sales,jp_sales,other_sales
title_id,platform_id,,,,
0,0,41.49,29.02,3.77,8.46
1,0,15.85,12.88,3.79,3.31
2,0,15.75,11.01,3.28,2.96
3,1,11.38,9.23,6.50,2.90
4,0,14.03,9.20,2.93,2.85
...,...,...,...,...,...
10662,5,0.01,0.00,0.00,0.00
2615,14,0.01,0.00,0.00,0.00
6031,4,0.00,0.00,0.00,0.00


In [ ]:
sales_df.dropna(subset=['title_id'])

### Juncture Table

In [20]:
#Create juncture table of title_id's and platform_id's as some titles can be on multiple platforms
#using game title table, create dictionary of game titles and id's for replacement in original vgdf with
#subset of columns game_title and platform

game_dict = game_title_df[['game_title']].to_dict('index')

for key in game_dict:
    game_dict[key] = game_dict[key]["game_title"]
    
game_dict_s = {value:key for key, value in game_dict.items()}
game_dict_s 


{'Wii Sports': 0,
 'Mario Kart Wii': 1,
 'Wii Sports Resort': 2,
 'New Super Mario Bros.': 3,
 'Wii Play': 4,
 'New Super Mario Bros. Wii': 5,
 'Nintendogs': 6,
 'Mario Kart DS': 7,
 'Wii Fit': 8,
 'Wii Fit Plus': 9,
 'Kinect Adventures!': 10,
 'Grand Theft Auto V': 11,
 'Grand Theft Auto: San Andreas': 12,
 'Brain Age: Train Your Brain in Minutes a Day': 13,
 'Pokemon Diamond/Pokemon Pearl': 14,
 'Grand Theft Auto: Vice City': 15,
 'Pokemon Ruby/Pokemon Sapphire': 16,
 'Pokemon Black/Pokemon White': 17,
 'Brain Age 2: More Training in Minutes a Day': 18,
 'Gran Turismo 3: A-Spec': 19,
 'Call of Duty: Modern Warfare 3': 20,
 'Call of Duty: Black Ops': 21,
 'Pokemon X/Pokemon Y': 22,
 'Call of Duty: Black Ops 3': 23,
 'Call of Duty: Black Ops II': 24,
 'Call of Duty: Modern Warfare 2': 25,
 'Grand Theft Auto III': 26,
 'Super Smash Bros. Brawl': 27,
 'Animal Crossing: Wild World': 28,
 'Mario Kart 7': 29,
 'Halo 3': 30,
 'Pokemon HeartGold/Pokemon SoulSilver': 31,
 'Super Mario 64': 32,

In [21]:
#Create new DF with just title and platform from original DF with older platforms removed

title_plat_df = vgdf_plat_clean[['Name','Platform']]
title_plat_df

,Name,Platform
0,Wii Sports,Wii
2,Mario Kart Wii,Wii
3,Wii Sports Resort,Wii
6,New Super Mario Bros.,DS
7,Wii Play,Wii
...,...,...
16593,Woody Woodpecker in Crazy Castle 5,GBA
16594,Men in Black II: Alien Escape,GC
16595,SCORE International Baja 1000: The Official Game,PS2
16596,Know How 2,DS


In [22]:
#referencing replacer dictionary from above
title_plat_replacer = {
    "Name": game_dict_s,
    "Platform": platformdict_s
}

title_plat_id = title_plat_df.replace(title_plat_replacer)
title_plat_id = title_plat_id.rename(columns={'Name':'title_id','Platform':'platform_id'})
title_plat_id

,title_id,platform_id
0,0,0
2,1,0
3,2,0
6,3,1
7,4,0
...,...,...
16593,10662,5
16594,2615,14
16595,6031,4
16596,10663,1


In [23]:
# title_plat_id_reduced = title_plat_id.drop_duplicates(subset=['title_id', 'platform_id'], keep='first')
# title_plat_id_reduced
title_plat_id_test = title_plat_id[title_plat_id.duplicated(subset=['title_id','platform_id'], keep=False)]
title_plat_id_test

,title_id,platform_id


## Review Helper

In [25]:
# created below tables to assist in formatting of review_df
review_helper = vgdf_plat_clean[['Name','Platform']]
review_helper["name_id"] = review_helper["Name"]
review_helper["platform_id"] = review_helper["Platform"]
review_helper

<ipython-input-25-c0207c2bcdd0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_helper["name_id"] = review_helper["Name"]
<ipython-input-25-c0207c2bcdd0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_helper["platform_id"] = review_helper["Platform"]


,Name,Platform,name_id,platform_id
0,Wii Sports,Wii,Wii Sports,Wii
2,Mario Kart Wii,Wii,Mario Kart Wii,Wii
3,Wii Sports Resort,Wii,Wii Sports Resort,Wii
6,New Super Mario Bros.,DS,New Super Mario Bros.,DS
7,Wii Play,Wii,Wii Play,Wii
...,...,...,...,...
16593,Woody Woodpecker in Crazy Castle 5,GBA,Woody Woodpecker in Crazy Castle 5,GBA
16594,Men in Black II: Alien Escape,GC,Men in Black II: Alien Escape,GC
16595,SCORE International Baja 1000: The Official Game,PS2,SCORE International Baja 1000: The Official Game,PS2
16596,Know How 2,DS,Know How 2,DS


In [26]:
review_helper = review_helper.replace(title_plat_replacer)
review_helper

,Name,Platform,name_id,platform_id
0,0,0,Wii Sports,Wii
2,1,0,Mario Kart Wii,Wii
3,2,0,Wii Sports Resort,Wii
6,3,1,New Super Mario Bros.,DS
7,4,0,Wii Play,Wii
...,...,...,...,...
16593,10662,5,Woody Woodpecker in Crazy Castle 5,GBA
16594,2615,14,Men in Black II: Alien Escape,GC
16595,6031,4,SCORE International Baja 1000: The Official Game,PS2
16596,10663,1,Know How 2,DS


In [27]:
review_helper.to_csv('review_helper.csv')

In [28]:
review_helper.loc[review_helper['Name']==190]

,Name,Platform,name_id,platform_id
252,190,4,Need for Speed: Most Wanted,PS2
498,190,3,Need for Speed: Most Wanted,PS3
1173,190,2,Need for Speed: Most Wanted,X360
1742,190,17,Need for Speed: Most Wanted,PSV
2005,190,10,Need for Speed: Most Wanted,XB
3585,190,14,Need for Speed: Most Wanted,GC
5900,190,11,Need for Speed: Most Wanted,PC
6149,190,15,Need for Speed: Most Wanted,WiiU
6278,190,1,Need for Speed: Most Wanted,DS
6492,190,5,Need for Speed: Most Wanted,GBA


## REVIEW DATA IMPORT

In [48]:
review_df = pd.read_csv("reviews_test.csv")
review_df.columns

Index(['Unnamed: 0', 'Name', 'Platform', 'Name_id', 'Platform_id',
       'Review Source', 'Review Score', 'Review Body'],
      dtype='object')

In [49]:
review_df.drop(columns=["Unnamed: 0", "Name", "Platform"], inplace=True)
review_df

,Name_id,Platform_id,Review Source,Review Score,Review Body
0,0,0,Official Nintendo Magazine UK,90,The ideal way to get into Wii gaming. It's fun...
1,0,0,GameZone,85,"Sure, it lacks goals and can be beaten very qu..."
2,0,0,GameDaily,80,Wii Sports certainly won't be hailed as the mo...
3,0,0,GamingExcellence,77,"Despite these two solo modes, Wii Sports is re..."
4,0,0,My Gamer,74,This is one you can feel confident about showi...
...,...,...,...,...,...
492,120,1,IGN,70,Cooking Mama does what it sets out to do very ...
493,120,1,Gaming Age,67,Whether you are a master in the kitchen or if ...
494,120,1,Nintendojo,64,"Controls are tight and responsive, and offer u..."
495,120,1,Modojo,60,The cheery 'bloop-de-bleep' repeated throughou...


In [52]:
review_df.rename(columns={"Name_id" : "title_id", "Platform_id" : "platform_id", "Review Source": "source",\
                 "Review Score" : "score", "Review Body" : "body"}, inplace=True)
review_df

,title_id,platform_id,source,score,body
0,0,0,Official Nintendo Magazine UK,90,The ideal way to get into Wii gaming. It's fun...
1,0,0,GameZone,85,"Sure, it lacks goals and can be beaten very qu..."
2,0,0,GameDaily,80,Wii Sports certainly won't be hailed as the mo...
3,0,0,GamingExcellence,77,"Despite these two solo modes, Wii Sports is re..."
4,0,0,My Gamer,74,This is one you can feel confident about showi...
...,...,...,...,...,...
492,120,1,IGN,70,Cooking Mama does what it sets out to do very ...
493,120,1,Gaming Age,67,Whether you are a master in the kitchen or if ...
494,120,1,Nintendojo,64,"Controls are tight and responsive, and offer u..."
495,120,1,Modojo,60,The cheery 'bloop-de-bleep' repeated throughou...


## DATA PUSH TO POSTGRES

In [29]:
rds_connection_string = f'{username}:{password}@localhost:5432/Video_Game_Sales_Review'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [53]:
vg_genre_df.to_sql(name='genre', con=engine, if_exists='append', index=True)

In [54]:
vg_platform_df.to_sql(name='platform', con=engine, if_exists='append', index=True)

In [55]:
vg_publisher_df.to_sql(name='publisher', con=engine, if_exists='append', index=True)

In [56]:
game_title_df.to_sql(name='title', con=engine, if_exists='append', index=True)

In [57]:
title_plat_id.to_sql(name='title_platform', con=engine, if_exists='append', index=False)

In [58]:
sales_df.to_sql(name='sales', con=engine, if_exists='append', index=True)

In [59]:
review_df.to_sql(name='review', con=engine, if_exists='append', index=False)